# Piezo Crystal Voltage Responce

In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import matplotlib.pyplot as plt 
import uncertainties as unc
import scipy as sci

from uncertainties import ufloat
from scipy.optimize import curve_fit
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
# Function Generator Voltage
FV = pd.read_csv('Data/F0002CH1.CSV', header=None, index_col=False, usecols=[3, 4])

# Piezo Voltage
PV = pd.read_csv('Data/F0002CH4.CSV', header=None, index_col=False, usecols=[3, 4])

# Photodetector Voltage 
DV = pd.read_csv('Data/F0002CH2.CSV', header=None, index_col=False, usecols=[3, 4])

In [3]:
# FV.rename(columns={'3':'Time (s)', '4':'Voltage (s)'}, inplace=True)
FV.columns = ['Time (s)', 'Voltage (V)']
PV.columns = ['Time (s)', 'Voltage (mV)']
DV.columns = ['Time (s)', 'Voltage (V)']

## Plot the Data

In [4]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

FV.plot(x = 'Time (s)', y = 'Voltage (V)' , ax = axes[0], legend=False,
        title = 'Voltage output of function generator',
        xlabel = 's',
        ylabel = 'V')

PV.plot(x = 'Time (s)', y = 'Voltage (mV)', ax = axes[1], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

DV.plot(x = 'Time (s)', y = 'Voltage (V)', ax = axes[2], legend=False,
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V')

axes[0].grid()
axes[1].grid()
axes[2].grid()
plt.tight_layout()
plt.show()

<Figure size 864x216 with 3 Axes>

## Clean up data

In [5]:
# Normalize photodetector intensity
DV['Intensity'] = (DV['Voltage (V)'] - np.min(DV['Voltage (V)'])) / (np.max(DV['Voltage (V)']) - np.min(DV['Voltage (V)']))

In [6]:
# Smoothing outputs 
# =================

# Hanning smoothing 
def smooth(x, window_len=11, window='hanning'):
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y[int((window_len-1)/2):-int((window_len-1)/2)]

HanIntFV, HanIntPV, HanIntDV, HanIntDVIntn = 21, 21, 21, 21 

FVHan = smooth(FV['Voltage (V)'], window_len=HanIntFV)
PVHan = smooth(PV['Voltage (mV)'], window_len=HanIntPV)
DVHan = smooth(DV['Voltage (V)'], window_len=HanIntDV)
DVHanIntn = smooth(DV['Intensity'], window_len=HanIntDVIntn)

## Curve fit the voltage output of photodetector

The curve for voltage over time:
$$
\begin{align*}
    V   &= a \sin^2(\nu \pi t + c\pi) + d + e t \\
    \nu &= b + a_2 \sin^2 (b_2 \pi t + c_2 \pi)
\end{align*}
$$

In [7]:
# Function
def DVFunc(x, a, b, c, d, e, a2, b2, c2):
    f = a2*np.sin(np.pi*b2*x + c2*np.pi)**2
    F = a*np.sin(np.pi*(b + f)*x + c*np.pi)**2 + d + e*x
    return F

def LinFunc(x, a, b):
    y = a*x + b
    return y

def LinFuncZero(x, a):
    y = a*x
    return y

# Functions to convert axis lables to n pi radians
def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))

In [8]:
# Introduce Cuts
# ==============
# Find positions of min and max values of PV
min_pos, max_pos = np.argmin(PV['Voltage (mV)']), np.argmax(PV['Voltage (mV)'])

Cut0Pos = min(min_pos, max_pos)
Cut1Pos = max(min_pos, max_pos)
Cut1 = PV['Voltage (mV)'][Cut1Pos:]

min_pos2, max_pos2 = np.argmin(Cut1)+Cut1Pos, np.argmax(Cut1)+Cut1Pos

Cut2Pos = max(min_pos2, max_pos2)

# Zero time
DV['Time Zeroed (s)'] = DV['Time (s)'] - DV['Time (s)'][Cut0Pos]
PV['Time Zeroed (s)'] = PV['Time (s)'] - PV['Time (s)'][Cut0Pos]

# Constrain DV and PV elements to those between min and max of PV
DVTime, DVVolt = DV['Time Zeroed (s)'][Cut0Pos:Cut1Pos], DVHan[Cut0Pos:Cut1Pos]
PVTime, PVVolt = PV['Time Zeroed (s)'][Cut0Pos:Cut1Pos], PVHan[Cut0Pos:Cut1Pos]

DVTime2, DVVolt2 = DV['Time Zeroed (s)'][Cut1Pos:Cut2Pos], DVHan[Cut1Pos:Cut2Pos]
PVTime2, PVVolt2 = PV['Time Zeroed (s)'][Cut1Pos:Cut2Pos], PVHan[Cut1Pos:Cut2Pos]

### Curve fit raw data

In [9]:
# Up to cut 1 
DVpopt, DVpcov = curve_fit(DVFunc, DVTime, DVVolt, 
#                            p0 = [0.5, 400, 1, -0.5, 0, 0, 0, 0], 
                           bounds = ([ 0.25, 300, -2, -1, -2, -np.inf,    0,       0], 
                                     [ 0.35, 800,  2,  0,  2,  np.inf,  100,  np.inf])
                          )

PVpopt, PVpcov = curve_fit(LinFunc, PVTime, PVVolt)

a1, b1, c1, d1, e1, a12, b12, c12 = DVpopt

In [10]:
# After cut 1
DVpopt2, DVpcov2 = curve_fit(DVFunc, DVTime2, DVVolt2, 
#                            p0 = [0.5, 1020, 1, -0.5, 0, 0, 0, 0], 
                             bounds = ([ 0.25,  200, -2, -1, -2, -np.inf,   0,       0], 
                                       [ 0.35,  800,  2,  0,  2,  np.inf, 100,  np.inf])
                            )

PVpopt2, PVpcov2 = curve_fit(LinFunc, PVTime2, PVVolt2)

a2, b2, c2, d2, e2, a22, b22, c22 = DVpopt2

In [11]:
# # Normalized Photodetector intensity
# DVIntn = DVHanIntn[Cut0Pos:Cut1Pos]/np.mean([a1, a2])
# DVIntn2 = DVHanIntn[Cut1Pos:Cut2Pos]/np.mean([a1, a2])

# # Convert time to space
# lam = 632 # (nm)

# DV['Displacement (nm)'] = (DV['Time (s)'] - DV['Time (s)'][Cut0Pos])*lam*np.mean([b1, b2])/2 
# DVDisp = DV['Displacement (nm)'][Cut0Pos:Cut1Pos]
# DVDisp2 = DV['Displacement (nm)'][Cut1Pos:Cut2Pos]

# # Curve Fit
# DVIpopt, DVIpcov = curve_fit(DVFunc, DVDisp, DVIntn, # method='dogbox',
#                              p0 = [0.42, 0.0026, 20, 0.5, 0,  0, 0, 0],
#                              bounds = ([ 0.4, -0.0025, -np.inf,  0.47, -10e-5,  -np.inf, -np.inf, -np.inf], 
#                                        [ 0.5,  0.0027,  np.inf,  0.53,  10e-5,   np.inf,  np.inf,  np.inf]) 
#                             )
# aI1, bI1, cI1, dI1, eI1, aI12, bI12, cI12 = DVIpopt

# DVIpopt2, DVIpcov2 = curve_fit(DVFunc, DVDisp2, DVIntn2, # method='dogbox',
#                              p0 = [0.42, 0.0026, 20, 0.5, 0,  0, 0, 0],
#                              bounds = ([ 0.4, -0.0025, -np.inf,  0.47, -10e-5,  -np.inf, -np.inf, -np.inf], 
#                                        [ 0.5,  0.0027,  np.inf,  0.53,  10e-5,   np.inf,  np.inf,  np.inf]) 
#                             )

# aI2, bI2, cI2, dI2, eI2, aI22, bI22, cI22 = DVIpopt2

## Plotting out the data

In [12]:
# Plot it out 
fig, ax = plt.subplots(2, 1, figsize = (8, 8), gridspec_kw={
                           'width_ratios': [2],
                           'height_ratios': [2, 3]})

# Voltage applied to piezo  per time
PV.plot(x = 'Time Zeroed (s)', y = 'Voltage (mV)', ax = ax[0], legend=False,
        title = 'Voltage applied to piezo crystal over time',
        xlabel = 's',
        ylabel = 'mV')

ax[0].plot(PVTime, LinFunc(PVTime, *PVpopt), c='orange', zorder=3, linestyle='dashed', 
           label = r'Fit: $V = ({0:.4g}) t  {1:+.4g}$'.format(*PVpopt))

ax[0].plot(PVTime2, LinFunc(PVTime2, *PVpopt2), c='red', zorder=3, linestyle='dashed',
           label = r'Fit: $V = ({0:.4g}) t  {1:+.4g}$'.format(*PVpopt2))

ax[0].plot(PV['Time Zeroed (s)'], PVHan, c='black',
           label = 'Hanning over {0} samples'.format(HanIntPV))

ax[0].legend()
ax[0].grid()

# Voltage output of photodetector per time
DV.plot(x = 'Time Zeroed (s)', y = 'Voltage (V)', legend=False, ax = ax[1],
        title = 'Voltage output of photodetector over time',
        xlabel = 's',
        ylabel = 'V',
        label = 'Data')

ax[1].plot(DVTime, DVFunc(DVTime, *DVpopt), c='orange', zorder=3, linestyle='dashed',
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi t {2:+.4g} \pi) {3:+.4g} {4:+.4g}t$' '\n' 
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi t {7:+.4g}\pi)$'
               .format(*DVpopt))

ax[1].plot(DVTime2, DVFunc(DVTime2, *DVpopt2), c='red', zorder=3, linestyle='dashed',
           
          label = 
               r'Fit: $V = {0:.4g} \sin^2(\nu \pi t {2:+.4g} \pi) {3:+.4g} {4:+.4g}t$' '\n'
               r'Where $\nu= {1:.4g} {5:+.4g} \sin^2 ({6:.4g} \pi t {7:+.4g}\pi)$'
               .format(*DVpopt2))

ax[1].plot(DV['Time Zeroed (s)'], DVHan, c='black', zorder=2,
           label = 'Hanning over {0} samples'.format(HanIntDV))

ax[1].set_ylim(np.min(DV['Voltage (V)'])*1.4)
ax[1].legend(loc='lower left')
ax[1].grid()

# Normalized intensity of photodetector with time
# DV.plot(x = 'Displacement (nm)', y = 'Intensity', legend=False, ax=ax[2],
#         title = 'Normalized output of photodetector',
#         xlabel = 'nm',
#         ylabel = 'V',
#         label = 'Data')

# ax[2].plot(DVDisp, DVFunc(DVDisp, *DVIpopt), c='orange', zorder=3,
#           label = 
#                r'Fit: $y = ({0:.2g}) \sin(\nu \pi x {2:+.2g} \pi) {3:+.2g} {4:+.2g}x$' '\n' 
#                r'Where $\nu= ({1:.3g}) {5:+.3g} \sin^2 ({6:.3g} \pi x {7:+.3g}\pi)$'
#                .format(aI1, bI1, cI1, dI1, eI1, aI12, bI12, cI12))

# ax[2].plot(DVDisp2, DVFunc(DVDisp2, *DVIpopt2), c='red', zorder=3,
#           label = 
#                r'Fit: $y = ({0:.2g}) \sin(\nu \pi x {2:+.2g} \pi) {3:+.2g} {4:+.2g}x$' '\n' 
#                r'Where $\nu= ({1:.3g}) {5:+.3g} \sin^2 ({6:.3g} \pi x {7:+.3g}\pi)$'
#                .format(aI2, bI2, cI2, dI2, eI2, aI22, bI22, cI22))

# ax[2].plot(DV['Displacement (nm)'], DVHanIntn, c='black', zorder=2,
#            label = 'Hanning over {0} samples'.format(HanIntDVIntn))

# ax[2].set_ylim(np.min(DV['Intensity']) - 0.55)
# ax[2].legend(loc='lower left')
# ax[2].grid()

plt.tight_layout()
plt.show()

<Figure size 576x576 with 2 Axes>

## Displacement of Piezo vs Voltage

### Plot phase change per voltage

In [13]:
def phase(x, B, C, A2, B2, C2):
    nu = A2*np.sin(np.pi*B2*x + C2*np.pi)**2
    phase = np.pi*(B + nu)*x + C2*np.pi
    return phase

def pickmid(x, n=0):
    mid = int(len(x)/2)
    MidElement = np.array(x)[mid + n]
    return MidElement

In [14]:
# Wavelength of laser
lam, unit = 632, 'nm'

phase1 = phase(DVTime, b1, c1, a12, b12, c12)
phase2 = phase(DVTime2, b2, c2, a22, b22, c22)
volt1 = LinFunc(PVTime, *PVpopt) - min(LinFunc(PVTime, *PVpopt))
volt2 = LinFunc(PVTime2, *PVpopt2) - min(LinFunc(PVTime2, *PVpopt2))

phase2cor = np.max(phase1) - (phase2 - np.min(phase2))

disp1 = phase1*lam/(4*np.pi)
disp2 = phase2cor*lam/(4*np.pi)

In [15]:
phases = np.concatenate((phase1, phase2cor), axis=None)
disps = np.concatenate((disp1, disp2), axis=None)
volts = np.concatenate((volt1, volt2), axis=None)

phasepopt, phasepcov = curve_fit(LinFuncZero, volts, phases)
disppopt, disppcov = curve_fit(LinFuncZero, volts, disps)

In [16]:
# Plot it out
# ===========
fig, ax  = plt.subplots(1, 2, figsize=(9, 4))

# Graph according to phase (n pi)
ax[0].plot(volt1, phase1, c='blue', label='Increasing voltage')
ax[0].plot(volt2, phase2cor, c='darkblue', label='Decreasing voltage')

# ax[0].arrow(pickmid(volt1), 
#             pickmid(phase1), 
#             pickmid(volt1) - pickmid(volt1, -10), 
#             pickmid(phase1) - pickmid(phase1, -10),
#             head_width = 2, length_includes_head=False, head_starts_at_zero=True)

# ax[0].arrow(pickmid(volt2), 
#             pickmid(phase2cor), 
#             pickmid(volt2) - pickmid(volt2, -10), 
#             pickmid(phase2cor) - pickmid(phase2cor, -10),
#             head_width = 2, length_includes_head=False)

# ax[0].scatter([pickmid(volt1), pickmid(volt1, -10), pickmid(volt2), pickmid(volt2, -10)], 
#               [pickmid(phase1), pickmid(phase1, -10), pickmid(phase2cor), pickmid(phase2cor, -10)])

ax[0].plot(volts, LinFuncZero(volts, *phasepopt), c='orange', zorder=3, linestyle=':',
           label = r'Fit: $\theta = {0:.4g} \pi V$'.format(*phasepopt/np.pi))

ax[0].yaxis.set_major_locator(plt.MultipleLocator(np.pi))
ax[0].yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 2))
ax[0].yaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))

ax[0].set_ylabel(r'$\theta$')
ax[0].set_xlabel('V')
ax[0].set_title(
    "\n".join(
        wrap('Change in phase of intensity per voltage applied to piezo crystal, assuming laser $\lambda = {0}$ {1}, with linear fit intersecting at the origin'.format(lam, unit), 50)
             )
        )
ax[0].legend(loc='upper left')
ax[0].grid()

# Graph according to displacement 

ax[1].plot(volt1, disp1, c='blue', label='Increasing voltage')
ax[1].plot(volt2, disp2, c='darkblue', label='Decreasing voltage')

# ax[1].arrow(pickmid(volt1), 
#             pickmid(disp1), 
#             pickmid(volt1) - pickmid(volt1, -10), 
#             (pickmid(disp1) - pickmid(disp1,-10)),
#             length_includes_head=False)

# ax[1].arrow(pickmid(volt2), 
#             pickmid(disp1), 
#             pickmid(volt2) - pickmid(volt2, -10), 
#             (pickmid(disp2) - pickmid(disp2, -10)),
#             length_includes_head=False)

# ax[1].scatter([pickmid(volt1), pickmid(volt1, -10), pickmid(volt2), pickmid(volt2, -10)], 
#               [pickmid(disp1), pickmid(disp1, -10), pickmid(disp2), pickmid(disp2, -10)])

ax[1].plot(volts, LinFuncZero(volts, *disppopt), c='orange', zorder=3, linestyle=':',
           label = r'Fit: $x = {0:.4g} V$'.format(*disppopt))

# loc = mp.ticker.MultipleLocator(base=316) 
# ax[1].xaxis.set_major_locator(loc)
ax[1].set_ylabel('Displacement ({0})'.format(unit))
ax[1].set_xlabel('V')
ax[1].set_title(
    "\n".join(
        wrap(r'Change in displacement per voltage applied to piezo crystal, assuming laser $\lambda = {0}$ {1}, with linear fit intersecting at the origin'.format(lam, unit), 50)
        )
    )
ax[1].grid()
ax[1].legend(loc='upper left')

plt.tight_layout()
plt.show()

<Figure size 648x288 with 2 Axes>